<a href="https://colab.research.google.com/github/JairEsc/Mat_Apl/blob/main/MN_T11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

In [3]:
#metodo de rayleigh

In [16]:
#metodo de iteracion en subespacio.
def gen_base_ortogonal(m,n):
    base=[]
    for j in range(m):
        b=[0]*n
        b[j]=1
        base.append(np.array(b))
    return np.array(base)
def factorizar_LU(matriz):#Codigo reciclado: Factorizacion LU
    n=len(matriz)
    L=np.zeros((n,n))
    U=np.zeros((n,n))
    for j in range(n):
        for i in range(j,n):
            sum=0
            for k in range(i):
                sum+=L[i][k]*U[k][j]
            L[i][j]=matriz[i][j]-sum
            if(i>j):
                sum=0
                for k in range(j):
                    sum+=L[j][k]*U[k][i]
                U[j][i]=(matriz[j][i]-sum)/L[j][j]
            else: 
                U[i][j]=1 
                if(L[i][i]==0):
                    print("Error en la construcción de L")
    return([L,U])

def superior(list,b):#Codigo reciclado:Resolver sistema triangular
  n=len(list)
  X=[b[n-1]/list[n-1][n-1]]
  X_inv=[X[0]]
  for i in range(1,n):#len(list) es el número de renglones de la matriz
    acum=0
    for j in range(i):#0<=j<=i-1 
      acum+=X[j]*list[n-i-1][n-1-j]
    X.append((b[n-1-i]-acum)/list[n-1-i][n-1-i])
    X_inv.insert(0,X[i])
  return X_inv

#Codigo reciclado:Resolver sistema triangular
def inferior(list,b):#list debe contener las entradas de la matriz triangular.
#Será aceptable introducir un arreglo de arreglos de distintos tamaños, para solo introducir las entradas no cero.
  X=[b[0]/list[0][0]]
  for i in range(1,len(list)):#len(list) es el número de renglones de la matriz
    acum=0
    for j in range(i):#0<=j<=i-1 
      acum+=X[j]*list[i][j]
    X.append((b[i]-acum)/list[i][i])
  return X
def solve(L_m,U_m,b):
    y=inferior(L_m,b)#Resolver Ly=b
    return(superior(U_m,y))#Resolver Ux=y
def check_diag(A,tol):#Pensado para matrices en general (NO simetricas)
    boolean=True
    breaker=False
    for i in range(len(A)):
        for j in range(i,len(A)):
            if(i!=j):
                if(abs(A[i][j])>tol):
                    boolean=False;
                    breaker=True;
                    break;
        if(breaker==True):
            break
    return(boolean)
def Jacobi_rot(A,tol,max_Iter,pr=True):
    count=0
    n=len(A)
    P=np.identity(n)
    while((check_diag(A,tol)==False)&(count<=max_Iter)):
        p,q=entrada_max(A)
        a=A[p][q]
        if(A[p][p]==A[q][q]):
            theta=np.pi/4
        else:
            theta=0.5*np.arctan(2*a/(A[p][p]-A[q][q]))
        #Ya tenemos la rotacion y podemos hacer el producto de matrices
        R=np.identity(n)
        R[p][p]=np.cos(theta)
        R[p][q]=-np.sin(theta)
        R[q][p]=-R[p][q]
        R[q][q]=R[p][p]
        A=np.dot(R.T,np.dot(A,R))#Se puede mejorar, modificando solo aquellas entradas que cambian.
        count+=1
        P=np.dot(P,R)
    if(count>max_Iter):
        print("Maxima iteracion alcanzada")
    else:
        if(pr==True):
            print("num. Iteraciones: ", str(count))
    return(P,np.diagonal(A))
def entrada_max(A):#Regresa una lista [i,j], tal que a_{i,j} es el valor maximo en valor absoluto.
#Pensado para matrices simetricas
    max=abs(A[0][1])
    index=[0,1]
    for i in range(len(A)):
        for j in range(i,len(A[0])):
            if(i!=j):
                if(max<abs(A[i][j])):
                    max=abs(A[i][j])
                    index=i,j
    return(index)
def min_eigen(A,tol,max_iter,pr=True,v_0=[]):
    n=len(A)
    if(len(v_0)==0):#Es opcional
        v_0=(np.zeros(n)+1/math.sqrt(n))#Normalizado
    L,U=factorizar_LU(A)
    v_1=solve(L,U,v_0)#Se resuelve el sistema
    lambda_0=np.dot(v_0,v_1)
    lambda_1=np.sqrt(np.dot(v_1,v_1))#se toma raíz para poder compararlos
    count=0
    while((abs(lambda_0-lambda_1)>tol)&(count<max_iter)):
        count+=1
        v_0=v_1/np.linalg.norm(v_1)#Se normaliza
        v_1=solve(L_m=L,U_m=U,b=v_0)#iteracion
        lambda_0=lambda_1
        lambda_1=np.dot(v_0,v_1)
    if(count>=max_iter):
        return("Maxima iteracion alcanzada")
    else:
        if(pr==True):
            print("numero de iteraciones: "+str(count))
        return([1/lambda_1,v_1])

In [5]:
##Algoritmo
##1 Genera subconjunto ortonormal.
## Usa ese vector inicial (con deflación para conservar la ortogonalidad) para UNA iteración de potencia inversa (o potencia.) 
#-> eso genera una nueva matriz \Phi de approximaciones de vectores. Se normalizan. Estos nuevos vectores no necesariamente son ortogonales (solo cuando A sea ortogonal).
## Hacer el producto B=\Phi.T*A*\Phi. Parar cuando esto sea diagonal.
## Aplicar Jacobi (Completo) sobre B.
#-> Genera un producto de matrices de rotacion Q, que puedo usar para actualizar.\Phi=\Phi_old*Q.
##Repetir

In [6]:
def quitar_contribucion(v_0,list_eigen):
    if list_eigen==[]:
        print("no se quito nada")
        return(np.array(v_0))
    else:
        aux=v_0
        for v in list_eigen:
            v_0-=np.dot(aux,np.array(v))*np.array(v)
        print("se quitaron: ",list_eigen)
        return v_0


In [7]:
np.dot([[2,0],[0,2]],np.array([1,2]))

array([2, 4])

In [41]:
def Iter_subespacio(A,m,tol=0.000001,max_iter=1000):
    n=len(A)
    phi_old=[[1/np.sqrt(3),1/np.sqrt(3),1/np.sqrt(3)],[-1/np.sqrt(2),1/np.sqrt(2),0]]
    print("phi_initial:," ,phi_old)
    print("Y es ortog,", np.dot(phi_old[0],phi_old[1]))
    B=A
    count=0;
    while(check_diag(B,tol=tol)==False):
        count+=1
        phi_new=[]
        for j in range(m):
            v_i=np.array(phi_old[j])
            if(j==0):
                v_i_next=min_eigen(A=A,tol=tol,max_iter=max_iter,pr=False,v_0=v_i)[1]
                phi_new.append(v_i_next/np.linalg.norm(v_i_next))
            if(j>0):
                aux=v_i
                for k in range(j):
                    v_i-=np.dot(aux,phi_new[k])*np.array(phi_new[k])
                print("wacha que si era ortogonal:",np.dot(v_i,phi_new[j-1]))
                v_i_next=np.linalg.solve(A,v_i)
                phi_new.append(v_i_next/np.linalg.norm(v_i_next))
        print("Pero defini phi_new",phi_new)
        print("La cual es ortogonal", np.dot(np.array(phi_new).T[0],np.array(phi_new).T[1]))
        B=np.dot(np.dot(phi_new,A),np.array(phi_new).T)
        print("no cambio phi",np.array(phi_old).T)
        #phi tiene renglones ortogonales.
        print("B no es d", B)
        phi_old=np.dot(np.array(phi_old).T,Jacobi_rot(B,tol,max_iter,pr=False)[0].T).T
        print("J es ortogonal?",Jacobi_rot(B,tol,max_iter,pr=False)[0])

        print("Acttualizo phi", phi_old)
        print("Checo que sigue siendo ortogonal:", phi_old[0],"*",phi_old[1],np.dot(phi_old[0],phi_old[1]))#No es :c
        
    print("Num. iteraciones", count)
    return (B)

In [12]:
P=Jacobi_rot([[4,-14,-12],[-14,10,13],[-12,13,1]],tol=0.00001,max_Iter=50)
np.dot(np.dot(P[0].T,[[4,-14,-12],[-14,10,13],[-12,13,1]]),P[0])


num. Iteraciones:  7


array([[-6.89202501e+00,  2.54932891e-15, -8.77293857e-15],
       [ 3.16732778e-15,  3.15356897e+01,  2.06425703e-08],
       [-9.18432492e-15,  2.06425679e-08, -9.64366467e+00]])

In [42]:
Iter_subespacio([[4,3,12],[3,10,13],[12,13,50]],m=2)

phi_initial:, [[0.5773502691896258, 0.5773502691896258, 0.5773502691896258], [-0.7071067811865475, 0.7071067811865475, 0]]
Y es ortog, 1.7796192843178415e-17
wacha que si era ortogonal: -1.978066325881488e-16
Pero defini phi_new [array([ 0.96864941,  0.03298568, -0.24623214]), array([-0.09738332,  0.96234231, -0.25379867])]
La cual es ortogonal -0.06176453522287375
no cambio phi [[ 0.57735027 -0.70710678]
 [ 0.57735027  0.70710678]
 [ 0.57735027  0.        ]]
B no es d [[ 1.05175165e+00 -1.05815103e-15]
 [-1.39465496e-15  6.20026000e+00]]
J es ortogonal? [[1. 0.]
 [0. 1.]]
Acttualizo phi [[ 0.57735027  0.57735027  0.57735027]
 [-0.70710678  0.70710678  0.        ]]
Checo que sigue siendo ortogonal: [0.57735027 0.57735027 0.57735027] * [-0.70710678  0.70710678  0.        ] 1.7796192843178415e-17
Num. iteraciones 1


array([[ 1.05175165e+00, -1.05815103e-15],
       [-1.39465496e-15,  6.20026000e+00]])

In [39]:
np.linalg.eig([[4,3,12],[3,10,13],[12,13,50]])

(array([56.74913984,  1.05175165,  6.19910851]),
 array([[ 0.22812092,  0.96865129,  0.09836429],
        [ 0.27441092,  0.03296734, -0.96104724],
        [ 0.93416245, -0.24622721,  0.25828796]]))